# Eager Execution で遊ぶ
いろいろ試してみる

In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

# 単純計算

In [3]:
x = tf.constant(1.)
y = tf.constant(2.)
x

<tf.Tensor: id=1, shape=(), dtype=float32, numpy=1.0>

In [4]:
for i in range(10):
    z = y
    y = x + y
    x = z
    print(y)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(13.0, shape=(), dtype=float32)
tf.Tensor(21.0, shape=(), dtype=float32)
tf.Tensor(34.0, shape=(), dtype=float32)
tf.Tensor(55.0, shape=(), dtype=float32)
tf.Tensor(89.0, shape=(), dtype=float32)
tf.Tensor(144.0, shape=(), dtype=float32)
tf.Tensor(233.0, shape=(), dtype=float32)


# パフォーマンス

In [5]:
import time
from tqdm import tqdm

In [6]:
def calc():
    X = tf.ones(shape=[256, 256], dtype=tf.float32)
    for i in range(100):
        X = tf.layers.dense(X, 256)
    return X

In [7]:
start = time.time()
with tf.Graph().as_default():
    X = calc()
    after_graph = time.time()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        result = sess.run(X)
end = time.time()
print('graph mode elapsed:', end - start)
print('graph mode run:', end - after_graph)
#print(result)

graph mode elapsed: 1.2025156021118164
graph mode run: 0.3529505729675293


In [8]:
start = time.time()
result = calc()
print('eager mode elapsed:', time.time() - start)
#print(result)

eager mode elapsed: 0.25362730026245117


In [9]:
start = time.time()
elapsed_list = []
with tf.Graph().as_default():
    X = calc()
    with tf.Session() as sess:
        for i in tqdm(range(50)):
            iter_start = time.time()
            sess.run(tf.global_variables_initializer())
            result = sess.run(X)
            elapsed_list.append(time.time() - iter_start)

print('graph mode elapsed:', time.time() - start)
print(' '.join([f'{elapsed:.3f}' for elapsed in elapsed_list]))
#print(result)

100%|██████████| 50/50 [00:12<00:00,  4.08it/s]

graph mode elapsed: 13.235857963562012
0.326 0.231 0.232 0.236 0.235 0.236 0.235 0.237 0.237 0.235 0.236 0.238 0.238 0.238 0.239 0.238 0.241 0.239 0.248 0.239 0.239 0.240 0.243 0.244 0.241 0.241 0.242 0.244 0.244 0.243 0.244 0.243 0.244 0.245 0.244 0.248 0.249 0.249 0.248 0.268 0.250 0.247 0.248 0.248 0.249 0.250 0.250 0.250 0.253 0.254


In [10]:
start = time.time()
elapsed_list = []

for i in tqdm(range(50)):
    iter_start = time.time()
    result = calc()
    elapsed_list.append(time.time() - iter_start)
print('eager mode elapsed:', time.time() - start)
print(' '.join([f'{elapsed:.3f}' for elapsed in elapsed_list]))
#print(result)

100%|██████████| 50/50 [00:21<00:00,  2.37it/s]

eager mode elapsed: 21.122700691223145
0.261 0.266 0.274 0.276 0.284 0.294 0.299 0.378 0.312 0.318 0.322 0.331 0.339 0.347 0.350 0.359 0.361 0.371 0.377 0.382 0.392 0.407 0.406 0.412 0.418 0.424 0.430 0.435 0.439 0.447 0.452 0.461 0.466 0.483 0.480 0.487 0.494 0.502 0.509 0.512 0.530 0.540 0.537 0.547 0.548 0.554 0.561 0.569 0.579 0.583
